In [ ]:
'''
The objective of this code is to convert a tif image to a cut segment 

'''

'\nThe objective of this code is to convert a tif image to a cut segment \n\n'

In [ ]:
!pip install rasterio

import rasterio
import rasterio.plot
import rasterio.features
import rasterio.warp
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import pprint
import glob
import os

In [ ]:
 !cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Setup the data folders to be cycled through
data_folder = '/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code/Datasets/MODIS_MCD43A4/21_11/'
year = '2004'
jdays = glob.glob(f'{data_folder}{year}/*')
files_band1, files_band4, files_band3 = [], [], [] 
for jday in jdays:
  files_band1.append(glob.glob(f'{jday}/*_B01.TIF')[0])
  files_band4.append(glob.glob(f'{jday}/*_B04.TIF')[0])
  files_band3.append(glob.glob(f'{jday}/*_B03.TIF')[0])


In [ ]:
def split_image(input_fname, rasterio_open_file, tl_corner_column, tl_corner_row, pixel_cut_length, output_directory):
  '''
  This function takes the rasterio open file and cuts it into a numpy array of 
  a particular size. 

  @param input_fname : [str] the (long) name of the input file containing the tile and date
  @param rasterio_open_file : [rasterio file] rastered imaged
  @param tl_corner_column : [int] column number of the top left corner that will be cut
  @param tl_corner_row : [int] row number of the top left corner that will be cut
  @param pixel_cut_length : [int] length of the cut in pixels. Note that in this
  code we will use the same length/width cut, but this can be changed. 
  @output_directory : [str] output directory for numpy array and images

  output:
  @cut_array : [np array] Numpy array containing RGB values for the cut segment
  @cut_array_img : [figure] Figure corresponding to cut array for sanity check

  '''
  # Find window parameters
  window = rasterio.windows.Window(tl_corner_column, tl_corner_row, pixel_cut_length, pixel_cut_length)

  # Open the file 
  subset = rasterio_open_file.read(window=window)
  
  subset = np.swapaxes(subset, axis1=0, axis2=1)
  subset = np.swapaxes(subset, axis1=1, axis2=2)

  # Old error
  # subset = np.swapaxes(subset, axis1=0, axis2=2)


  # print(np.shape(subset))

  # Create a filename based on the window
  filename = f'{input_fname}_{tl_corner_column}_{tl_corner_row}_{pixel_cut_length}'

  # Save the numpy array for the subset
  np.save(f'{output_directory}np_arrays/{filename}.npy', subset, allow_pickle=True)

  # Save a figure (for troubleshooting)
  fig1 = plt.figure(figsize=(6,8.5))
  plt.imshow(subset[:,:,0])
  # rasterio.plot.show(subset)
  plt.title(f'Band 4 Subset\n{window}')
  plt.xlabel('Column #')
  plt.ylabel('Row #')
  plt.savefig(f'{output_directory}images/{filename}.png')
  plt.close(fig1)
  print(f'Saved file {filename}...')

In [ ]:
# Cycle through all of the files
files_num = len(files_band1)
for files_ind in np.arange(files_num):
# for files_ind in np.arange(20):
  file_list = [files_band1[files_ind], files_band4[files_ind], files_band3[files_ind]]

  # Get the name of the individual file
  fname = os.path.basename(files_band1[files_ind]).split('_')[0]

  jday_folder = files_band1[files_ind].split('/')[-2]
  output_dir =  f'/content/gdrive/Shared drives/2020_FDLUSA_Earth Science_Knowledge Discovery Framework/Code/Datasets/training_set_tmp2/{jday_folder}/'
  if not os.path.exists(output_dir):
    os.mkdir(output_dir)
  if not os.path.exists(f'{output_dir}images'):
    os.mkdir(f'{output_dir}images')
  if not os.path.exists(f'{output_dir}np_arrays'):
    os.mkdir(f'{output_dir}np_arrays')

  # for ind in np.arange(len(file_list)):
  with rasterio.open(file_list[0]) as src0:
      meta = src0.meta

  # Update meta to reflect the number of layers
  meta.update(count = len(file_list))

  # Read each layer and write it to stack
  with rasterio.open(f'{output_dir}stack.tif', 'w', **meta) as dst:
      for id, layer in enumerate(file_list, start=1):
          with rasterio.open(layer) as src1:
              dst.write_band(id, src1.read(1))
  src = rasterio.open(f'{output_dir}stack.tif')

  band_1 = src.read(1)
  np.shape(band_1)

  subset_size = 448 # pixels
  corner_values = []

  column_values = np.arange(0, np.shape(band_1)[1], subset_size)
  row_values = np.arange(0, np.shape(band_1)[0], subset_size)

  for column_val in column_values:
    for row_val in row_values:
      split_image(fname, src, column_val, row_val, subset_size, output_dir)

  os.remove(f'{output_dir}stack.tif')




(500, 500, 3)
Saved file MCD43A4.A2004073.h21v11.006.2016112221845_0_0_500...
(500, 500, 3)
Saved file MCD43A4.A2004073.h21v11.006.2016112221845_0_500_500...
(500, 500, 3)
Saved file MCD43A4.A2004073.h21v11.006.2016112221845_0_1000_500...
(500, 500, 3)
Saved file MCD43A4.A2004073.h21v11.006.2016112221845_0_1500_500...
(500, 400, 3)
Saved file MCD43A4.A2004073.h21v11.006.2016112221845_0_2000_500...
(500, 500, 3)
Saved file MCD43A4.A2004073.h21v11.006.2016112221845_500_0_500...
(500, 500, 3)
Saved file MCD43A4.A2004073.h21v11.006.2016112221845_500_500_500...
(500, 500, 3)
Saved file MCD43A4.A2004073.h21v11.006.2016112221845_500_1000_500...
(500, 500, 3)
Saved file MCD43A4.A2004073.h21v11.006.2016112221845_500_1500_500...
(500, 400, 3)
Saved file MCD43A4.A2004073.h21v11.006.2016112221845_500_2000_500...
(500, 500, 3)
Saved file MCD43A4.A2004073.h21v11.006.2016112221845_1000_0_500...
(500, 500, 3)
Saved file MCD43A4.A2004073.h21v11.006.2016112221845_1000_500_500...
(500, 500, 3)
Saved file